In [22]:
from langchain.schema import Document
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
# from langchain_community.vectorstores import Chroma
from langchain_chroma import Chroma

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

from dotenv import load_dotenv
load_dotenv()

True

In [23]:
embedding_function = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

docs = [
    Document(
        page_content="Peak Performance Gym was founded in 2015 by former Olympic athlete Marcus Chen. With over 15 years of experience in professional athletics, Marcus established the gym to provide personalized fitness solutions for people of all levels. The gym spans 10,000 square feet and features state-of-the-art equipment.",
        metadata={"source": "about.txt"}
    ),
    Document(
        page_content="Peak Performance Gym is open Monday through Friday from 5:00 AM to 11:00 PM. On weekends, our hours are 7:00 AM to 9:00 PM. We remain closed on major national holidays. Members with Premium access can enter using their key cards 24/7, including holidays.",
        metadata={"source": "hours.txt"}
    ),
    Document(
        page_content="Our membership plans include: Basic (₹1,500/month) with access to gym floor and basic equipment; Standard (₹2,500/month) adds group classes and locker facilities; Premium (₹4,000/month) includes 24/7 access, personal training sessions, and spa facilities. We offer student and senior citizen discounts of 15% on all plans. Corporate partnerships are available for companies with 10+ employees joining.",
        metadata={"source": "membership.txt"}
    ),
    Document(
        page_content="Group fitness classes at Peak Performance Gym include Yoga (beginner, intermediate, advanced), HIIT, Zumba, Spin Cycling, CrossFit, and Pilates. Beginner classes are held every Monday and Wednesday at 6:00 PM. Intermediate and advanced classes are scheduled throughout the week. The full schedule is available on our mobile app or at the reception desk.",
        metadata={"source": "classes.txt"}
    ),
    Document(
        page_content="Personal trainers at Peak Performance Gym are all certified professionals with minimum 5 years of experience. Each new member receives a complimentary fitness assessment and one free session with a trainer. Our head trainer, Neha Kapoor, specializes in rehabilitation fitness and sports-specific training. Personal training sessions can be booked individually (₹800/session) or in packages of 10 (₹7,000) or 20 (₹13,000).",
        metadata={"source": "trainers.txt"}
    ),
    Document(
        page_content="Peak Performance Gym's facilities include a cardio zone with 30+ machines, strength training area, functional fitness space, dedicated yoga studio, spin class room, swimming pool (25m), sauna and steam rooms, juice bar, and locker rooms with shower facilities. Our equipment is replaced or upgraded every 3 years to ensure members have access to the latest fitness technology.",
        metadata={"source": "facilities.txt"}
    )
]

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
db = Chroma.from_documents(docs, embedding_function)
retriever = db.as_retriever(search_type="mmr", search_kwards = {"k":3})

In [24]:
retriever.invoke("Who found the gym and what are the timigs?")

[Document(id='57fb01ba-3bd2-4c9d-9971-dac17f7c0e3b', metadata={'source': 'about.txt'}, page_content='Peak Performance Gym was founded in 2015 by former Olympic athlete Marcus Chen. With over 15 years of experience in professional athletics, Marcus established the gym to provide personalized fitness solutions for people of all levels. The gym spans 10,000 square feet and features state-of-the-art equipment.'),
 Document(id='4002850e-daf2-41de-a131-7d17e41d0e8f', metadata={'source': 'membership.txt'}, page_content='Our membership plans include: Basic (₹1,500/month) with access to gym floor and basic equipment; Standard (₹2,500/month) adds group classes and locker facilities; Premium (₹4,000/month) includes 24/7 access, personal training sessions, and spa facilities. We offer student and senior citizen discounts of 15% on all plans. Corporate partnerships are available for companies with 10+ employees joining.'),
 Document(id='5c7a0147-b759-43e6-8c48-0bc065f9500c', metadata={'source': 'cl

In [41]:
template ="""
Answer the question based on following context: {context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [42]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = prompt | llm



In [43]:
from typing import Annotated, Literal, TypedDict
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from pydantic import BaseModel, Field


In [44]:
class AgentState(TypedDict):
    messages: list[BaseMessage]
    documents: list[Document]
    on_topic: str


In [29]:
class GradeQuestion(BaseModel):
    """ Boolean value to check whether a question is related to the Peak Performance Gym """

    score: str = Field(
        description="Question is about gym? If yes -> 'Yes' if not -> 'No' "
    )

def question_classifier(state: AgentState):
    question = state["messages"][-1].content
    system = """ You are a classifier that determines whether a user's question is about one of the following topics

    1. Gym History & Founder
    2. Operating Hours
    3. Membership Plans
    4. Fitness Classes
    5. Personal Trainers
    6. Facilities & Equipment

    If the question IS about any of these topics, respond with 'Yes'. Otherwise, respond with 'No'.

    """
    grade_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system),
            ("human", "User question: {question}")
        ]
    )

    structured_llm = llm.with_structured_output(GradeQuestion)
    graded_llm = grade_prompt | structured_llm
    result = graded_llm.invoke({"question": question})

    state["on_topic"] = result.score

    return state


In [45]:
def on_topic_router(state: AgentState) -> str:
    if state["on_topic"] == "Yes":
        return "on_topic"
    else:
        return "off_topic"

def retrieve(state: AgentState) -> AgentState:
    question = state["messages"][-1].content
    state["documents"] = retriever.invoke(question)
    return state

def generated_answer(state: AgentState):
    documents = state["documents"]
    question = state["messages"][-1].content
    generation = rag_chain.invoke({
        "context": documents, "question": question})
    state["messages"].append(generation)

def off_topic_response(state: AgentState):
    state["messages"].append(AIMessage(content="I'm sorry! I cannot answer this question!"))
    return state


In [46]:
from langgraph.graph import StateGraph, END

graph = StateGraph(AgentState)

CLASSIFIER = "classifier"
OFF_TOPIC = "off_topic_response"
RETRIEVE = "retrieve"
GENERATED_ANSWER = "generated_answer"

graph.add_node(CLASSIFIER, question_classifier)
graph.add_node(OFF_TOPIC, off_topic_response)
graph.add_node(RETRIEVE, retrieve)
graph.add_node(GENERATED_ANSWER, generated_answer)

graph.add_conditional_edges(CLASSIFIER, on_topic_router, {
    "on_topic": RETRIEVE,
    "off_topic": OFF_TOPIC
})

graph.add_edge(RETRIEVE, GENERATED_ANSWER)
graph.add_edge(GENERATED_ANSWER, END)
graph.add_edge(OFF_TOPIC, END)

graph.set_entry_point(CLASSIFIER)

app = graph.compile()


In [47]:
print(app.get_graph().draw_mermaid())

%%{init: {'flowchart': {'curve': 'linear'}}}%%
graph TD;
	__start__([<p>__start__</p>]):::first
	classifier(classifier)
	off_topic_response(off_topic_response)
	retrieve(retrieve)
	generated_answer(generated_answer)
	__end__([<p>__end__</p>]):::last
	__start__ --> classifier;
	generated_answer --> __end__;
	off_topic_response --> __end__;
	retrieve --> generated_answer;
	classifier -. &nbsp;on_topic&nbsp; .-> retrieve;
	classifier -. &nbsp;off_topic&nbsp; .-> off_topic_response;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [49]:
app.invoke(input={
    "messages": [HumanMessage(content="Who is the owner and what are the timings?")]
})

{'messages': [HumanMessage(content='Who is the owner and what are the timings?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Peak Performance Gym was founded by **Marcus Chen**.\n\nThe gym's timings are:\n*   **Monday through Friday:** 5:00 AM to 11:00 PM\n*   **Weekends:** 7:00 AM to 9:00 PM\n*   The gym is closed on major national holidays, but members with Premium access can enter 24/7, including holidays.", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-019b212f-b40c-7203-8954-2c0c7dcec609-0', usage_metadata={'input_tokens': 564, 'output_tokens': 88, 'total_tokens': 932, 'input_token_details': {'cache_read': 0}})],
 'documents': [Document(id='70c44123-6600-41e2-ade7-f4b0cca30ffb', metadata={'source': 'hours.txt'}, page_content='Peak Performance Gym is open Monday through Friday from 5:00 AM to 11:00 PM. On weekends, our hours are 7:00 AM to 9:00 PM. 

In [34]:
app.invoke(input={
    "messages": [HumanMessage(content="Is young will and Stranger thing season 5 will same? are they both Noah schnapp?")]
})

{'messages': [HumanMessage(content='Is young will and Stranger thing season 5 will same? are they both Noah schnapp?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="I'm sorry! I cannot answer this question!", additional_kwargs={}, response_metadata={})],
 'on_topic': 'No'}